In [1]:
#Import Relevant Libraries
import random
import numpy as np
import cvxpy as cp

from qiskit import QuantumCircuit, execute, Aer, IBMQ, BasicAer
from qiskit.visualization import plot_histogram, plot_state_qsphere as plot_q
from qiskit.quantum_info import Statevector

from qiskit import quantum_info as qi
from toqito.channel_metrics import channel_fidelity

from toqito.state_metrics import fidelity
from toqito.channels import partial_trace as pt
from qiskit.quantum_info import DensityMatrix, state_fidelity, partial_trace

In [4]:
'''
Defines parameters that specify the purification of state to be tested. Create a QuantumCircuit object that
creates the purification on (size) number of qubits.
'''
unitary_param_1 = [3.24383098, 3.50187279, 1.26248366, 1.99354817, 4.28031102, 5.99851666, 3.03466457, 4.21065453]

size = 2
layers = 2
state = QuantumCircuit(size)
for j in range(layers):
    for i in range(size):
        state.rx(unitary_param_1[j*size*2 + i], i)
        state.ry(unitary_param_1[j*2*size + i + size], i)

    for i in range(size-1):
        state.cx(i, i+1)

state.draw('mpl')

In [56]:
'''
Extract the matrix representation of the state \rho_S
'''
psi = np.array(DensityMatrix.from_instruction(state.reverse_bits()))
rho = pt(psi, [2], [2, 2])

[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]


In [2]:
'''
Define matrix representations of required gates.
'''
zero = np.array([[1, 0], [0, 0]], dtype=complex)
one = np.array([[0, 0], [0, 1]], dtype=complex)
I = np.array([[1, 0], [0, 1]], dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

In [3]:
'''
Create list of group elements.
'''
lis = []
lis.append(np.kron(zero, I) + np.kron(one, I))
lis.append(np.kron(zero, I) + np.kron(one, -I))
lis.append(np.kron(zero, I) + np.kron(one, -1j*X))
lis.append(np.kron(zero, I) + np.kron(one, 1j*X))
lis.append(np.kron(zero, I) + np.kron(one, -1j*Y))
lis.append(np.kron(zero, I) + np.kron(one, 1j*Y))
lis.append(np.kron(zero, I) + np.kron(one, -1j*Z))
lis.append(np.kron(zero, I) + np.kron(one, 1j*Z))

In [57]:
'''
SDP to calculate the Fidelity. Refer to Section 5.
'''
shape = rho.shape

X = cp.Variable(shape, complex=True)
omega = cp.Variable(np.kron(I, I).shape, complex=True)
G = cp.bmat([[rho, X.H],[X, pt(omega, [2], [2, 2])]])

c = [omega>>0, G>>0, cp.real(cp.trace(omega))==1]

'''
Add the constraint to ensure that sig is G-Symmetric extendible
'''
for item in lis:
    c.append(omega@item == item@omega)

obj = cp.Maximize(cp.real(cp.trace(X)))
problem = cp.Problem(obj, constraints = c)
rootFid = problem.solve()
print("Root Fidelity is : " + str(rootFid))

WARN: A->p (column pointers) not strictly increasing, column 20 empty
WARN: A->p (column pointers) not strictly increasing, column 25 empty
Root Fidelity is : 0.7071060195870931


In [58]:
print("Fidelity is : " + str(rootFid**2))

Fidelity is : 0.49999892293630244
